In [1]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [4]:
#buildint the training dataset
block_size = 3
X, Y = [], []

for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [5]:
Y.shape

torch.Size([228146])

In [6]:
C = torch.randn((27,2))
emb = C[X]

In [7]:
emb

tensor([[[ 1.0198, -0.1765],
         [ 1.0198, -0.1765],
         [ 1.0198, -0.1765]],

        [[ 1.0198, -0.1765],
         [ 1.0198, -0.1765],
         [-1.1763, -0.9313]],

        [[ 1.0198, -0.1765],
         [-1.1763, -0.9313],
         [-0.1680, -0.0854]],

        ...,

        [[ 0.5798, -0.0704],
         [ 0.5798, -0.0704],
         [-0.5882,  0.4887]],

        [[ 0.5798, -0.0704],
         [-0.5882,  0.4887],
         [ 0.5798, -0.0704]],

        [[-0.5882,  0.4887],
         [ 0.5798, -0.0704],
         [ 1.6391, -0.5878]]])

In [8]:
torch.cat([emb[:,0,:], emb[:,1,:], emb[:,2,:]], 1)


tensor([[ 1.0198, -0.1765,  1.0198, -0.1765,  1.0198, -0.1765],
        [ 1.0198, -0.1765,  1.0198, -0.1765, -1.1763, -0.9313],
        [ 1.0198, -0.1765, -1.1763, -0.9313, -0.1680, -0.0854],
        ...,
        [ 0.5798, -0.0704,  0.5798, -0.0704, -0.5882,  0.4887],
        [ 0.5798, -0.0704, -0.5882,  0.4887,  0.5798, -0.0704],
        [-0.5882,  0.4887,  0.5798, -0.0704,  1.6391, -0.5878]])

In [9]:
len(torch.unbind(emb, 1))

3

In [10]:
torch.cat(torch.unbind(emb, 1),1)

tensor([[ 1.0198, -0.1765,  1.0198, -0.1765,  1.0198, -0.1765],
        [ 1.0198, -0.1765,  1.0198, -0.1765, -1.1763, -0.9313],
        [ 1.0198, -0.1765, -1.1763, -0.9313, -0.1680, -0.0854],
        ...,
        [ 0.5798, -0.0704,  0.5798, -0.0704, -0.5882,  0.4887],
        [ 0.5798, -0.0704, -0.5882,  0.4887,  0.5798, -0.0704],
        [-0.5882,  0.4887,  0.5798, -0.0704,  1.6391, -0.5878]])

In [12]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)
h = torch.tanh(emb.view(32,6) @ W1 + b1)
h


RuntimeError: shape '[32, 6]' is invalid for input of size 1368876

In [13]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100,generator = g)
W2 = torch.randn((100,27), generator = g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [14]:
sum(p.nelement() for p in parameters)

3481

In [15]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,Y)


In [16]:
for p in parameters:
    p.requires_grad = True

In [17]:
for _ in range(10000):
    # minibatch
    ix = torch.randint(0, X.shape[0], (32,))

    # forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 +b1)
    logits = h@ W2 +b2
    loss = F.cross_entropy(logits,Y[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.1 * p.grad

print(loss.item())

2.464995861053467


In [18]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 +b1)
logits = h@ W2 +b2
loss = F.cross_entropy(logits,Y)
loss.item()

2.4899680614471436